In [4]:
import argparse
import os
import pathlib
import pickle
from lib2to3.pgen2.tokenize import tokenize

import accelerate
import datasets
import evaluate
import numpy as np
import torch
import tqdm
import wandb
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
device = 'cuda'
model='Llama-2-7b-hf'
data_dir = 'c/Users/nikos/PycharmProjects/UncertainLLMs/data'
hf_datasets_cache = 'mnt/c/Users/nikos/.cache/huggingface/datasets'
output_dir = '/c/Users/nikos/PycharmProjects/UncertainLLMs/data'

model = AutoModelForCausalLM.from_pretrained(f"meta-llama/{model}",
                                             torch_dtype=torch.float16,
                                             cache_dir=hf_datasets_cache).cuda()




In [23]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

tokenizer_config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 776/776 [00:00<?, ?B/s]
C:\Users\nikos\.conda\envs\UncertainLLMs\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nikos\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In

In [24]:
dataset = datasets.load_from_disk('C:\\Users\\nikos\\PycharmProjects\\UncertainLLMs\\data\\squad_v2\\squad2_dataset')
print('Loaded squad2 dataset')

Loaded squad2 dataset


In [25]:
train_dataset = dataset.train_test_split(test_size=(1 - 0.1), seed=123)['train']
questions = train_dataset

In [27]:
dataloader = torch.utils.data.DataLoader(questions, batch_size=1)
period_token_id = tokenizer('. ')['input_ids'][1]
eos_tokens = ['Question:', ' Question:', '\n', 'Answer:', ' Answer:', 'Q:']
question_framing_ids = [[tokenizer(eos_token)['input_ids'][1]] for eos_token in eos_tokens]
squad_metric = evaluate.load("squad")
rouge = evaluate.load('rouge')
exact_match_metric = evaluate.load("exact_match")

In [33]:
if dataset == 'squad2':
    print(dataset)
else: print('false')

with torch.no_grad():
    max_length_of_generated_sequence = 256
    sequences = []
    for batch in tqdm.tqdm(dataloader):
        input_ids = torch.cat(batch['id']).to(device).reshape(
            1, -1) if dataset == 'squad2' else batch['input_ids'].to(device)

false


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1187/1187 [00:01<00:00, 1023.99it/s]


In [29]:
most_likely_generation = model.generate(input_ids,
                                        num_beams=5,
                                        num_return_sequences=2,
                                        do_sample=False,
                                        max_length=input_ids.shape[1] +
                                        max_length_of_generated_sequence,
                                        eos_token_id=period_token_id,
                                        bad_words_ids=question_framing_ids)

C:\Users\nikos\.conda\envs\UncertainLLMs\lib\site-packages\transformers\generation\configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\nikos\.conda\envs\UncertainLLMs\lib\site-packages\transformers\generation\configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
